In [32]:
from glob import glob
import json
import os

files = glob('ultrachat-astroawani/*.json')
files = sorted(files, key = lambda x: os.path.getmtime(x))
len(files)

60533

In [18]:
texts = []
for f in files:
    try:
        with open(f) as fopen:
            data = json.load(fopen)[1:]
        for d in data:
            texts.append(d['content'])
    except:
        pass
        
texts = [t for t in texts if t]
texts = [t for t in texts if len(t)]
texts = list(set(texts))
len(texts)

418410

In [34]:
import re

rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'Kode',
    'kode',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik',
    'jawaban',
    'teknis',
    'berkualitas',
    'mencoba',
    'kampanye',
    'komunitas',
    'stabilitas',
    'Stabilitas',
    'metode',
    'pria',
    'butuh',
    'jadwal',
    'kasus',
    'otomatis',
    'populer',
    'bisnis',
    'probabilitas',
    'rusak',
    'kapasitas',
    'rutinitas'
]
rejected_words = set(rejected_words)

def check_indon(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    return len(set(string.split()) & rejected_words) > 0

In [20]:
with open('astroawani-malay.texts', 'w') as fopen:
    for t in texts:
        fopen.write(f'{json.dumps(t)}\n')

In [21]:
!cp astroawani-malay.texts ~/ssd3/ctranslate2

In [31]:
mapping = {}
for f in glob('/home/husein/ssd3/ctranslate2/astroawani-malay.texts*.splitted.requested'):
    with open(f) as fopen:
        for l in fopen:
            try:
                l = json.loads(l)
                mapping[l['src']] = l['r']
            except:
                pass
        
len(mapping)

418409

In [35]:
with open('ultrachat-astroawani-malay.jsonl', 'w') as fopen_l:
    for f in files:
        with open(f) as fopen:
            data = json.load(fopen)
        if not data:
            continue
        for i in range(0, len(data), 1):
            data[i]['content_ms'] = mapping.get(data[i]['content'])
            indon = False
            if data[i]['content'] and check_indon(data[i]['content']):
                indon = True
            data[i]['indon'] = indon
        fopen_l.write(f'{json.dumps(data)}\n')